In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from os import listdir
from os.path import isfile, join
import os
import shutil
import urllib
from datetime import datetime

In [ ]:
url = 'https://www.atlasobscura.com/places?page={}&sort=likes_count'
result = requests.get(url)

print(result)
print(result.text)

In [ ]:
soup = BeautifulSoup(result.text)

In [ ]:
mydivs = soup.find_all("a", {"class": "content-card content-card-place"})
'https://www.atlasobscura.com' + mydivs[1]['href']

In [ ]:
with open('places_url.txt', 'w') as f:
    for i in range(1,401):
        url = 'https://www.atlasobscura.com/places?page={}&sort=likes_count'.format(i)
        result = requests.get(url)
        soup = BeautifulSoup(result.text)
        mydivs = soup.find_all("a", {"class": "content-card content-card-place"})
        for anchor in mydivs:
            f.write('https://www.atlasobscura.com' + anchor['href'] + "\n")
    f.close();

In [ ]:
def saveHtmlFiles():
    notDownloadedUrls = []
    with open('places_url.txt') as file:
        for url in file:
            try:
                urllib.request.urlretrieve(url,url.split('/')[-1].replace('\n','')+'.html')
            except Exception as e:
                notDownloadedUrls.append(url)
                continue
    file.close()
    return notDownloadedUrls

In [ ]:
notDownloadedUrls = saveHtmlFiles()

while notDownloadedUrls:
    with open('places_url.txt', 'w') as f:
        for url in notDownloadedUrls:
            f.write(url + "\n")
    f.close();
    notDownloadedUrls = saveHtmlFiles()

In [7]:
file_path = os.getcwd()
files = []
with open('places_url.txt') as file:
    for url in file:
        files.append(url.split('/')[-1].replace('\n','')+'.html')
    file.close()
file_list = np.array_split(files,400)


In [ ]:
for i in range(400):
    directory = 'page_'+ str(i+1)

    target_dir = os.path.join(file_path, directory)

    os.mkdir(target_dir)

    for file_name in file_list[i]:
        shutil.move(os.path.join(file_path, file_name), target_dir)

In [ ]:
for i in range(1,401):
    dir = os.getcwd() + '\page_{}'.format(i)
    files = [f for f in listdir(dir) if isfile(join(dir, f)) and f.lower().endswith(('.html'))]
    for file in files:
        with open(file) as fp:
            soup = BeautifulSoup(fp, "html.parser")
            soup.text

In [139]:

with open('.\page_1\catacombes-de-paris.html', 'rb+') as fp:
    soup = BeautifulSoup(fp, "html.parser")
    placeName = ('' if soup.select('h1')[0].text.strip() is None else soup.select('h1')[0].text.strip())
    placeTags = '' if soup.find('div', {'class': 'item-tags'}) is None else [a.text.lstrip() for a in soup.find('div', {'class': 'item-tags'}).select('a', {'class': 'itemTags__link'})]
    numPeopleVisited = ('' if soup.find('aside', {'class': 'DDPage__item-actions'}) is None else soup.find('aside', {'class': 'DDPage__item-actions'}).select('div', {'class': 'title-md item-action-count'})[3].text)
    numPeopleWant = ('' if soup.find('aside', {'class': 'DDPage__item-actions'}) is None else soup.find('aside', {'class': 'DDPage__item-actions'}).select('div', {'class': 'title-md item-action-count'})[4].text)
    numPeopleWant = '' if numPeopleVisited == '' else int(''.join(filter(str.isdigit, numPeopleVisited)))
    placeDesc = '' if soup.find('div', {'id': 'place-body'}) is None else soup.find('div', {'id': 'place-body'}).findNext().text
    placeShortDesc = soup.select('h3', {'class': 'DDPage__header-dek'})
    placesNearby = '' if soup.findAll('div', {'class': 'DDPageSiderailRecirc__item-title'}) is None else set([a.text for a  in soup.findAll('div', {'class': 'DDPageSiderailRecirc__item-title'})])
    placeAddress = '' if soup.find('address', {'class': 'DDPageSiderail__address'}) is None else soup.find('address', {'class': 'DDPageSiderail__address'}).text.strip()
    placeAlt = '' if soup.find('div', {'class': 'DDPageSiderail__coordinates'}) is None else soup.find('div', {'class': 'DDPageSiderail__coordinates'}).text.strip().split(',')[0]
    placeLong = '' if soup.find('div', {'class': 'DDPageSiderail__coordinates'}) is None else soup.find('div', {'class': 'DDPageSiderail__coordinates'}).text.strip().split(',')[1]
    placeEditors = '' if soup.findAll('a', {'class': 'DDPContributorsList__contributor'}) is None else [ a.text.lstrip() for a in soup.findAll('a', {'class': 'DDPContributorsList__contributor'})]
    placePubDate = '' if soup.find('div', {'class': 'DDPContributor__name'}) is None else datetime.strptime(soup.find('div', {'class': 'DDPContributor__name'}).text,'%B %d, %Y').date()
    placeRelatedLists = '' if soup.findAll('div', {'class': 'CardRecircSection'}) is None else ([a.findNext('span').text for a in soup.findAll('div', {'class': 'CardRecircSection'})[2].findNext('div', {'class': 'CardRecircSection__card-grid'}).findAll('a')])
    placeRelatedPlaces = '' if soup.findAll('div', {'class': 'CardRecircSection'}) is None else ([a.findNext('span').text for a in soup.findAll('div', {'class': 'CardRecircSection'})[1].findNext('div', {'class': 'CardRecircSection__card-grid'}).findAll('a')])
    placeUrl = ('' if soup.find('div', {'class': 'DDPageSiderail__website'}) is None else soup.find('div', {'class': 'DDPageSiderail__website'}).find('a')['href'].strip())